CROP VIDEO

In [36]:
# check wanted crop dimensions via 1. frame of video
import cv2
import os

# parameters
x0, x1, y0, y1 = 0, 1, 0, 1
input, video_format = 'Auerbacher', '.mp4'


input_file = input + video_format
video = cv2.VideoCapture(input_file)
_, image = video.read()
cv2.imwrite(f'{input}_frame0.png', image)

img = cv2.imread(f'{input}_frame0.png')
print(f'dimensions: {img.shape[1]} x {img.shape[0]} px')

crop_img = img[y0:y1, x0:x1]
cv2.imshow(f'{input}_cropped', crop_img)
cv2.waitKey(0)
os.remove(f'{input}_frame0.png')

dimensions: 320 x 576 px


In [28]:
# crop video with dimensions from above
import subprocess
import moviepy.editor as mpy

crop_params = f'crop={x1-x0}:{y1-y0}:{(x0+x1)/2}:{(y0+y1)/2}'
output_file = input + '_cropped' + video_format
ffmpeg_command = f'ffmpeg -i {input_file} -filter:v {crop_params} {output_file}'

subprocess.run(ffmpeg_command, shell=True)

# check changes
clip = mpy.VideoFileClip(input_file)
print(f'INPUT : {clip.size} px, {clip.reader.nframes} frames, {clip.duration}s, {round(clip.reader.nframes/clip.duration, 3)} fps')

clip = mpy.VideoFileClip(output_file)
print(f'OUTPUT: {clip.size} px, {clip.reader.nframes} frames, {clip.duration}s, {round(clip.reader.nframes/clip.duration, 3)} fps')


INPUT : [320, 576] px, 402 frames, 13.38s, 30.045 fps
OUTPUT: [120, 276] px, 402 frames, 13.38s, 30.045 fps
